In [1]:
%run init.ipynb

matchzoo version 0.0.1
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [2]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 2118/2118 [00:00<00:00, 9007.52it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 18841/18841 [00:04<00:00, 4038.04it/s]
Building Vocabulary from a datapack.: 100%|██████████| 1614998/1614998 [00:00<00:00, 3746407.13it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 2118/2118 [00:00<00:00, 9141.49it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 18841/18841 [00:04<00:00, 4135.85it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:09<00:00, 2093.37it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 1

In [3]:
preprocessor.context

{'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x136469d68>,
 'vocab_size': 9645,
 'embedding_input_dim': 9645}

In [4]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [5]:
padding_callback = mz.models.DSSM.get_default_padding_callback()

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=1,
    num_neg=4
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=32,
    stage='train',
    resample=True,
    callback=padding_callback
)

testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=32,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.DSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['vocab_size']
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

DSSM(
  (mlp_left): Sequential(
    (0): Sequential(
      (0): Linear(in_features=9645, out_features=300, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=300, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (mlp_right): Sequential(
    (0): Sequential(
      (0): Linear(in_features=9645, out_features=300, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=300, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (out): Sequential(
    (0): Linear(in_features=1, out_fea

In [11]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [12]:
trainer.run()

[Iter-32 Loss-1.531]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4894 - normalized_discounted_cumulative_gain@5(0.0): 0.559 - mean_average_precision(0.0): 0.5129



[Iter-64 Loss-1.260]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.425 - normalized_discounted_cumulative_gain@5(0.0): 0.4917 - mean_average_precision(0.0): 0.4565



[Iter-96 Loss-1.180]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3711 - normalized_discounted_cumulative_gain@5(0.0): 0.4602 - mean_average_precision(0.0): 0.4314



[Iter-128 Loss-1.093]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3759 - normalized_discounted_cumulative_gain@5(0.0): 0.4409 - mean_average_precision(0.0): 0.4293



[Iter-160 Loss-1.048]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3565 - normalized_discounted_cumulative_gain@5(0.0): 0.4212 - mean_average_precision(0.0): 0.4073



[Iter-192 Loss-1.019]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3882 - normalized_discounted_cumulative_gain@5(0.0): 0.4599 - mean_average_precision(0.0): 0.4415



[Iter-224 Loss-1.009]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3835 - normalized_discounted_cumulative_gain@5(0.0): 0.4567 - mean_average_precision(0.0): 0.4327



[Iter-256 Loss-0.957]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.407 - normalized_discounted_cumulative_gain@5(0.0): 0.4696 - mean_average_precision(0.0): 0.4417



[Iter-288 Loss-0.942]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3433 - normalized_discounted_cumulative_gain@5(0.0): 0.4197 - mean_average_precision(0.0): 0.3998



[Iter-320 Loss-0.920]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3651 - normalized_discounted_cumulative_gain@5(0.0): 0.4332 - mean_average_precision(0.0): 0.4174

Cost time: 710.622061252594s


In [12]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=5, use_multiprocessing=False)

Epoch 1/20
32/32 [==============================] - 7s 215ms/step - loss: 1.3325
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4431849853601904 - normalized_discounted_cumulative_gain@5(0.0): 0.5295386323998266 - mean_average_precision(0.0): 0.48303488812718776
Epoch 2/20
32/32 [==============================] - 6s 176ms/step - loss: 1.3159
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4353814849661657 - normalized_discounted_cumulative_gain@5(0.0): 0.5032525911610362 - mean_average_precision(0.0): 0.4776049822282439
Epoch 3/20
32/32 [==============================] - 5s 171ms/step - loss: 1.2955
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4088637099689691 - normalized_discounted_cumulative_gain@5(0.0): 0.48351010067595823 - mean_average_precision(0.0): 0.4432379861560312
Epoch 4/20
32/32 [==============================] - 6s 173ms/step - loss: 1.2726
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.46569627211992487 - normalized_d